In [1]:
import numpy as np
import pandas as pd
from scipy import stats

from matplotlib import pyplot as plt
from matplotlib.patches import Patch

In [2]:
time = 365
inc = 28
inf = 7

In [3]:
def create_hh():
    hh_size = np.random.choice([3, 4, 5, 6], size = 340, replace = True)
    hh_size = hh_size[np.cumsum(hh_size) < 1000]
    
    leftover = 1000 - hh_size.sum()
    if leftover < 3:
        # Randomly sample leftover amount indices from hh_size, and add one
        hh = np.arange(len(hh_size))
        idx = np.random.choice(hh[hh_size < 6], size = leftover, replace = False)
        hh_size[idx] += 1
    else:
        hh_size = np.append(hh_size, leftover)
    return hh_size

In [4]:
def SEIR(beta_H, beta_C, inc, inf):
    hh_size = create_hh()
    
    data = pd.DataFrame({'ID': range(1000),
                         'SIZE': np.repeat(hh_size, repeats = hh_size),
                         'HH': np.repeat(range(len(hh_size)), repeats = hh_size),
                         'S': np.append(0, np.ones(999)),
                         'E': np.append(1, np.zeros(999)),
                         'E_count': np.append(1, np.zeros(999)),
                         'I': np.zeros(1000),
                         'I_count': np.zeros(1000),
                         'R': np.zeros(1000),
                         'INC': np.append(np.round(stats.norm.rvs(inc, 2)), np.zeros(999)),
                         'INF': np.zeros(1000)
                        })
    
    results = data.loc[:, 'ID':'HH']
    results['TYPE'] = np.nan
    results.loc[0, 'TYPE'] = '0'
    results['TIME'] = np.nan
    results.loc[0, 'TIME'] = 0
    
    for t in range(time):
        if t % 10 == 0: print(t, end = ' ')
        recovered = (data['INF'] > 0) & (data['I_count'] == data['INF'])
        if sum(recovered) > 0:
            data.loc[recovered, 'R'] = 1
            data.loc[recovered, 'I'] = 0

            # Why is this count set to 0 once they recover?
            data.loc[recovered, 'I_count'] = 0

        new_inf = (data['INC'] > 0) & (data['E_count'] == data['INC'])
        num_new_inf = sum(new_inf)

        if num_new_inf > 0:
            # Why do new infections have SD = 1?
            random_inf = np.round(stats.norm.rvs(inf, 1, size = num_new_inf))
            data.loc[new_inf, 'I'] = 1
            data.loc[new_inf, 'INF'] = random_inf
            data.loc[new_inf, 'E'] = 0

            # Why is this set to 0?
            data.loc[new_inf, 'E_count'] = 0

        I_H = data.groupby('HH').sum()['I']
        summary = pd.DataFrame({'I_H': I_H, 
                                'I_C': data.sum()['I'] - I_H
                               })
        dd = data[['HH', 'S']].copy()

        dd['I_H'] = dd.apply(lambda x: summary.loc[x['HH'], 'I_H'], axis = 1)
        dd['I_C'] = dd.apply(lambda x: summary.loc[x['HH'], 'I_C'], axis = 1)

        risk_H = dd['S'] * beta_H * dd['I_H'] / 1000
        risk_C = dd['S'] * beta_C * dd['I_C'] / 1000

        new_inf_H = stats.binom.rvs(1, risk_H, size = data.shape[0])
        new_inf_C = stats.binom.rvs(1, risk_C, size = data.shape[0])
        new_exposed = (new_inf_H == 1) | (new_inf_C == 1)

        num_new_exposed = sum(new_exposed)
        if num_new_exposed > 0:
            data.loc[new_exposed, 'E'] = 1
            # Why is this calculated only upon exposure?
            random_inc = np.round(stats.norm.rvs(inc, 2, size = num_new_exposed))
            data.loc[new_exposed, 'E'] = 1
            data.loc[new_exposed, 'INC'] = random_inc

            results['TYPE'].where(~((new_inf_H == 1) & (new_inf_C == 1)) | ~pd.isna(results['TYPE']), 'B', inplace = True)
            results['TYPE'].where(~(new_inf_H == 1) | ~pd.isna(results['TYPE']), 'H', inplace = True)
            results['TYPE'].where(~(new_inf_C == 1) | ~pd.isna(results['TYPE']), 'C', inplace = True)
            results['TIME'].where(~(new_exposed == 1) | ~pd.isna(results['TIME']), t, inplace = True)
        
        data.loc[data['E'] == 1, 'E_count'] += 1
        data.loc[data['I'] == 1, 'I_count'] += 1
        data.loc[data['E'] == 1, 'S'] = 0

    return results
        

In [5]:
def metrics(results):
    state = results[~pd.isna(results['TIME'])]
    incidence = state.shape[0]/1000
    
    prop_hh = np.nan
    if incidence != 0:
        prop_hh = state[state['TYPE'] == 'H'].shape[0] / state.shape[0]
    
    return incidence, prop_hh

In [6]:
def mse(results, target):
    return ((np.array(metrics(results)) - target)**2).sum()

def bce(results, target):
    incidence, prop_hh = metrics(results)
    
    incidence_loss = target[0] * np.log(incidence + eps) + (1 - target[0]) * np.log(1 - incidence + eps)
    prop_hh_loss = target[1] * np.log(prop_hh + eps) + (1 - target[1]) * np.log(1 - prop_hh + eps)
    return incidence_loss + prop_hh_loss

# 10 February

In [ ]:
idcs = np.load('results/archive/idcs_3.npy')
prps = np.load('results/archive/prps_3.npy')

beta_C_vals = np.load('results/archive/beta_C_vals_3.npy')
beta_H_vals = np.load('results/archive/beta_H_vals_3.npy')
cc, hh = np.meshgrid(beta_C_vals, beta_H_vals)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (9.5, 3.5))

plot_hh = axs[0].scatter(idcs.flatten(), prps.flatten(), c = hh.flatten(), alpha = 0.5)
axs[0].set_xlabel(r'$I$')
axs[0].set_ylabel(r'$p_{\rm HH}$')
plt.colorbar(plot_hh, ax = axs[0])

axs[0].scatter(0.05, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.05, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.05, 0.75, marker = 'x', c = 'red', s = 100)

axs[0].scatter(0.1, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.1, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.1, 0.75, marker = 'x', c = 'red', s = 100)

axs[0].scatter(0.3, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.3, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.3, 0.75, marker = 'x', c = 'red', s = 100)
axs[0].set_title(r'$\beta_H$', loc = 'left')

plot_cc = axs[1].scatter(idcs.flatten(), prps.flatten(), c = cc.flatten(), alpha = 0.5)
axs[1].set_xlabel(r'$I$')
axs[1].set_ylabel(r'$p_{\rm HH}$')
plt.colorbar(plot_cc, ax = axs[1])

axs[1].scatter(0.05, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.05, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.05, 0.75, marker = 'x', c = 'red', s = 100)

axs[1].scatter(0.1, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.1, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.1, 0.75, marker = 'x', c = 'red', s = 100)

axs[1].scatter(0.3, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.3, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.3, 0.75, marker = 'x', c = 'red', s = 100)
axs[1].set_title(r'$\beta_C$', loc = 'left')

plt.savefig('plots/prp_vs_inc_1.png', transparent = True, dpi = 300)

In [ ]:
idx = (idcs >= 0.09) & (idcs <= 0.11) & (prps >= 0.2) & (prps <= 0.3)

In [ ]:
hh[idx], cc[idx]

In [ ]:
hh

In [ ]:
cc

In [ ]:
results = SEIR(23.8, 0.22, num_weeks_inf, num_weeks_inc)

In [ ]:
metrics(results)

## More Results

In [ ]:
idcs = np.load('results/1/idcs.npy')
prps = np.load('results/1/prps.npy')

cc = np.load('results/1/cc.npy')
hh = np.load('results/1/hh.npy')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (9.5, 3.5))

plot_hh = axs[0].scatter(idcs.flatten(), prps.flatten(), 
                         c = np.repeat(hh.flatten(), 5), alpha = 0.5)
axs[0].set_xlabel(r'$I$')
axs[0].set_ylabel(r'$p_{\rm HH}$')
axs[0].set_aspect('equal')
fig.colorbar(plot_hh, ax = axs[0])
axs[0].set_title(r'$\beta_H$', loc = 'left')
axs[0].scatter(0.05, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.05, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.05, 0.75, marker = 'x', c = 'red', s = 100)

axs[0].scatter(0.1, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.1, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.1, 0.75, marker = 'x', c = 'red', s = 100)

axs[0].scatter(0.3, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.3, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.3, 0.75, marker = 'x', c = 'red', s = 100)

plot_cc = axs[1].scatter(idcs.flatten(), prps.flatten(), 
                         c = np.repeat(cc.flatten(), 5), alpha = 0.5)
axs[1].set_xlabel(r'$I$')
axs[1].set_ylabel(r'$p_{\rm HH}$')
axs[1].set_aspect('equal')
fig.colorbar(plot_hh, ax = axs[1])
axs[1].set_title(r'$\beta_C$', loc = 'left')
axs[1].scatter(0.05, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.05, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.05, 0.75, marker = 'x', c = 'red', s = 100)

axs[1].scatter(0.1, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.1, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.1, 0.75, marker = 'x', c = 'red', s = 100)

axs[1].scatter(0.3, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.3, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.3, 0.75, marker = 'x', c = 'red', s = 100)

plt.savefig('prp_vs_inc_2.png', transparent = True, dpi = 300)

# 14 February

Results for a $\beta_H$ grid of $[20, 20.8, 21.6, \ldots, 39.2, 40]$ and $\beta_C$ grid of $[0, 0.04, 0.08, \ldots, 0.96, 1]$.

In [ ]:
idcs = np.load('results/2/idcs.npy')
prps = np.load('results/2/prps.npy')

cc = np.load('results/2/cc.npy')
hh = np.load('results/2/hh.npy')

In [ ]:
mean_idcs = idcs.mean(axis = 2)
mean_prps = prps.mean(axis = 2)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (11, 3.5))

plot_hh = axs[0].scatter(mean_idcs.flatten(), mean_prps.flatten(), 
                         c = hh.flatten(), alpha = 0.5)

axs[0].set_xlabel(r'$I$')
axs[0].set_ylabel(r'$p_{\rm HH}$')
axs[0].set_aspect('equal')
fig.colorbar(plot_hh, ax = axs[0])
axs[0].set_title(r'$\beta_H$', loc = 'left')
axs[0].scatter(0.05, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.05, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.05, 0.75, marker = 'x', c = 'red', s = 100)

axs[0].scatter(0.1, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.1, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.1, 0.75, marker = 'x', c = 'red', s = 100)

axs[0].scatter(0.3, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.3, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.3, 0.75, marker = 'x', c = 'red', s = 100)

plot_cc = axs[1].scatter(mean_idcs.flatten(), mean_prps.flatten(), 
                         c = cc.flatten(), alpha = 0.5)
axs[1].set_xlabel(r'$I$')
axs[1].set_ylabel(r'$p_{\rm HH}$')
axs[1].set_aspect('equal')
fig.colorbar(plot_hh, ax = axs[1])
axs[1].set_title(r'$\beta_C$', loc = 'left')
axs[1].scatter(0.05, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.05, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.05, 0.75, marker = 'x', c = 'red', s = 100)

axs[1].scatter(0.1, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.1, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.1, 0.75, marker = 'x', c = 'red', s = 100)

axs[1].scatter(0.3, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.3, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.3, 0.75, marker = 'x', c = 'red', s = 100)

plt.savefig('plots/prp_vs_inc_2.png', transparent = True, dpi = 300)

Results for a $\beta_H$ grid of $[10, 12, 14, \ldots, 48, 50]$ and $\beta_C$ grid of $[0, 0.1, 0.2, \ldots, 1.9, 2]$.

In [ ]:
idcs = np.load('results/3/idcs_3.npy')
prps = np.load('results/3/prps_3.npy')

cc = np.load('results/3/cc_3.npy')
hh = np.load('results/3/hh_3.npy')

In [ ]:
mean_idcs = idcs.mean(axis = 2)
mean_prps = prps.mean(axis = 2)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (11, 3.5))

plot_hh = axs[0].scatter(mean_idcs.flatten(), mean_prps.flatten(), 
                         c = hh.flatten(), alpha = 0.5)
axs[0].set_xlabel(r'$I$')
axs[0].set_ylabel(r'$p_{\rm HH}$')
axs[0].set_aspect('equal')
fig.colorbar(plot_hh, ax = axs[0])
axs[0].set_title(r'$\beta_H$', loc = 'left')
axs[0].scatter(0.05, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.05, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.05, 0.75, marker = 'x', c = 'red', s = 100)

axs[0].scatter(0.1, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.1, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.1, 0.75, marker = 'x', c = 'red', s = 100)

axs[0].scatter(0.3, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.3, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.3, 0.75, marker = 'x', c = 'red', s = 100)

plot_cc = axs[1].scatter(mean_idcs.flatten(), mean_prps.flatten(), 
                         c = cc.flatten(), alpha = 0.5)
axs[1].set_xlabel(r'$I$')
axs[1].set_ylabel(r'$p_{\rm HH}$')
axs[1].set_aspect('equal')
fig.colorbar(plot_hh, ax = axs[1])
axs[1].set_title(r'$\beta_C$', loc = 'left')
axs[1].scatter(0.05, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.05, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.05, 0.75, marker = 'x', c = 'red', s = 100)

axs[1].scatter(0.1, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.1, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.1, 0.75, marker = 'x', c = 'red', s = 100)

axs[1].scatter(0.3, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.3, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.3, 0.75, marker = 'x', c = 'red', s = 100)

plt.savefig('plots/prp_vs_inc_3.png', transparent = True, dpi = 300)

# 15 February

In [ ]:
idcs_1 = np.load('results/4/idcs_1.npy')
prps_1 = np.load('results/4/prps_1.npy')

idcs_2 = np.load('results/4/idcs_2.npy')
prps_2 = np.load('results/4/prps_2.npy')

idcs_3 = np.load('results/4/idcs_3.npy')
prps_3 = np.load('results/4/prps_3.npy')

beta_H_vals = [20.6, 30, 29.6]
beta_C_vals = [0.30, 0.12, 0.20]

In [ ]:
fig, axs = plt.subplots(3, 2, figsize = (10, 16))
bins = np.linspace(0, 1, 51)

axs[0][0].hist(idcs_1, bins = bins)
axs[0][1].hist(prps_1, bins = bins)
axs[0][0].set_xlim(0, 1)
axs[0][1].set_xlim(0, 1)
axs[0][0].set_title(r'$\beta_H = 20.6$, $\beta_C = 0.30$', loc = 'left')
axs[0][0].set_xlabel('Incidence')
axs[0][1].set_xlabel('Proportion HH')

axs[1][0].hist(idcs_2, bins = bins)
axs[1][1].hist(prps_2, bins = bins)
axs[1][0].set_xlim(0, 1)
axs[1][1].set_xlim(0, 1)
axs[1][0].set_title(r'$\beta_H = 30$, $\beta_C = 0.12$', loc = 'left')
axs[1][0].set_xlabel('Incidence')
axs[1][1].set_xlabel('Proportion of Infections from Household')

axs[2][0].hist(idcs_3, bins = bins)
axs[2][1].hist(prps_3, bins = bins)
axs[2][0].set_xlim(0, 1)
axs[2][1].set_xlim(0, 1)
axs[2][0].set_title(r'$\beta_H = 29.6$, $\beta_C = 0.20$', loc = 'left')
axs[2][0].set_xlabel('Incidence')
axs[2][1].set_xlabel('Proportion of Infections from Household')
plt.savefig('plots/param_check.png', transparent = False, dpi = 300)

In [ ]:
idcs_1.mean(), prps_1.mean()

In [ ]:
idcs_2.mean(), prps_2.mean()

In [ ]:
idcs_3.mean(), prps_3.mean()

In [ ]:
idcs_3[prps_3 == 1]

In [ ]:
prps_1[idcs_1 == 0]

In [ ]:
prps_2[idcs_2 == 0]

In [ ]:
prps_3[idcs_3 == 0]

# 28 February

In [ ]:
def metrics(results):
    state = results[~pd.isna(results['TIME'])]
    i = state.shape[0]/1000
    
    #p_hh = np.nan
    #if i != 0:
    #    p_hh = state[state['TYPE'] == 'H'].shape[0] / state.shape[0]
    #return i, p_hh
    
    sar = np.nan
    if i != 0:
        num_primary = np.sum(results.groupby('HH')['TIME'].sum() > 0) # households that were infected
        idx = results.groupby('HH')['TYPE'].apply(lambda x: ~np.all(x.isna()))
        num_contact = (results.groupby('HH')['SIZE'].sum()[idx]**(1/2)).sum() # total people in all those households
        sar = state[(state['TYPE'] == 'H') | (state['TYPE'] == 'B')].shape[0] / (num_contact - num_primary)
    
    return i, sar

In [ ]:
chain = np.load('results/5/chain_2.npy')

In [ ]:
from matplotlib.collections import LineCollection

x = chain[:, 0]
y = chain[:, 1]
cs = np.linspace(0,1,len(x))

points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

fig, ax = plt.subplots()
lc = LineCollection(segments, cmap='viridis')
lc.set_array(cs)
lc.set_linewidth(1)
line = ax.add_collection(lc)
ax.autoscale()
plt.xlabel(r'$\beta_H$')
plt.ylabel(r'$\beta_C$')
# Plot loss value over iterations
# Deliverables
# Update MCMC (by Wednesday)
# - save likelihood
# - each iteration averages over 100 iterations (NUM_ITER)
# Implement SAR (by Friday)
# Test R code on local device

In [ ]:
best = np.load('results/5/best_2.npy')

In [ ]:
beta_H, beta_C = best

In [ ]:
idcs = np.load('results/6/idcs_5.npy')
sars = np.load('results/6/sars_5.npy')

cc = np.load('results/6/cc_5.npy')
hh = np.load('results/6/hh_5.npy')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (11, 3.5))

plot_hh = axs[0].scatter(idcs.flatten(), sars.flatten(), 
                         c = hh.flatten(), alpha = 0.5)
axs[0].set_xlabel(r'$I$')
axs[0].set_ylabel(r'$\rm SAR$')
axs[0].set_aspect('equal')
fig.colorbar(plot_hh, ax = axs[0])
axs[0].set_title(r'$\beta_H$', loc = 'left')
axs[0].scatter(0.05, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.1, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.3, 0.25, marker = 'x', c = 'red', s = 100)

plot_cc = axs[1].scatter(idcs.flatten(), sars.flatten(), 
                         c = cc.flatten(), alpha = 0.5)
axs[1].set_xlabel(r'$I$')
axs[1].set_ylabel(r'$\rm SAR$')
axs[1].set_aspect('equal')
fig.colorbar(plot_cc, ax = axs[1])
axs[1].set_title(r'$\beta_C$', loc = 'left')
axs[1].scatter(0.05, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.1, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.3, 0.25, marker = 'x', c = 'red', s = 100)

plt.savefig('plots/prp_vs_sar.png', transparent = True, dpi = 300)

In [ ]:
cc[(idcs > 0.05) & (idcs < 0.15)]

In [ ]:
hh[(idcs > 0.05) & (idcs < 0.15)]

In [ ]:
cc[(idcs > 0.25) & (idcs < 0.35)]

In [ ]:
hh[(idcs > 0.25) & (idcs < 0.35)]

In [ ]:
cc[(idcs > 0.01) & (idcs < 0.07)]

In [ ]:
hh[(idcs > 0.01) & (idcs < 0.07)]

In [ ]:
beta_Hs = np.linspace(5, 35, 61)
beta_Cs = np.linspace(0, 1, 21)

In [ ]:
beta_Hs

In [ ]:
i = 1, j = 0

In [ ]:
for i in range(3):
    for j in range(4):
        for k in range(5):
            print(20*i + 5*j + k)

In [ ]:
%%time
SEIR(beta_H, beta_C, inc, inf)

In [ ]:
beta_H, beta_C

In [ ]:
n = 25
sars = [None]*n
idcs = [None]*n
for i in range(n):
    print(i, end = ' ')
    results = SEIR(beta_H, beta_C, inc, inf)
    idc, sar = metrics(results)
    idcs[i] = idc
    sars[i] = sar
    print()

In [ ]:
np.mean(idcs)

In [ ]:
plt.hist(idcs, alpha = 0.5)

In [ ]:
np.mean(sars)

In [ ]:
plt.hist(sars, alpha = 0.5)

In [ ]:
chain

In [ ]:
np.unique(chain).shape

In [ ]:
np.prod(chain.shape)

# 6 March 

In [ ]:
best = np.load('results/8/best.npy')
liks = np.load('results/8/liks.npy')

In [ ]:
best

In [ ]:
liks[:, 0]

In [ ]:
# How many simulations per parameter set: Pick 5 parameter sets and check them on different simulation sizes (10, 50, 100, 250, 500, 750, 1000)
# Sophia will test on Winton
# Paralellize code
# MCMC fits to stochastic models: harder to find the global optimum. Initial guess makes a big difference.
# Grid search to get better initial guess.

In [ ]:
chain

In [ ]:
plt.plot(chain[:, 0], chain[:, 1])

In [ ]:
plt.plot(liks[:, 0])

In [ ]:
chain = np.load('results/8/chain.npy')

In [ ]:
chain

# 15 March

## Grid Search

In [ ]:
idcs = np.load('results/9/idcs.npy')
sars = np.load('results/9/sars.npy')

cc = np.load('results/9/cc.npy')
hh = np.load('results/9/hh.npy')

In [ ]:
idcs.shape

In [ ]:
np.sum(np.isnan(idcs))

In [ ]:
sars

In [ ]:
mean_idcs = idcs.mean(axis = 2)
mean_sars = sars.mean(axis = 2)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (11, 3.5))

plot_hh = axs[0].scatter(mean_idcs.flatten(), mean_sars.flatten(), 
                         c = hh.flatten(), alpha = 0.5)

axs[0].set_xlabel(r'$I$')
axs[0].set_ylabel(r'${\rm SAR}$')
axs[0].set_aspect('equal')
fig.colorbar(plot_hh, ax = axs[0])
axs[0].set_title(r'$\beta_H$', loc = 'left')
axs[0].scatter(0.05, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.05, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.05, 0.75, marker = 'x', c = 'red', s = 100)

axs[0].scatter(0.1, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.1, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.1, 0.75, marker = 'x', c = 'red', s = 100)

axs[0].scatter(0.3, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.3, 0.5, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.3, 0.75, marker = 'x', c = 'red', s = 100)

plot_cc = axs[1].scatter(mean_idcs.flatten(), mean_sars.flatten(), 
                         c = cc.flatten(), alpha = 0.5)
axs[1].set_xlabel(r'$I$')
axs[1].set_ylabel(r'${\rm SAR}$')
axs[1].set_aspect('equal')
fig.colorbar(plot_hh, ax = axs[1])
axs[1].set_title(r'$\beta_C$', loc = 'left')
axs[1].scatter(0.05, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.05, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.05, 0.75, marker = 'x', c = 'red', s = 100)

axs[1].scatter(0.1, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.1, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.1, 0.75, marker = 'x', c = 'red', s = 100)

axs[1].scatter(0.3, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.3, 0.5, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.3, 0.75, marker = 'x', c = 'red', s = 100)

plt.savefig('plots/sar_vs_idc.png', transparent = True, dpi = 300)

In [ ]:
def create_hh():
    hh_size = np.random.choice([3, 4, 5, 6], size = 340, replace = True)
    hh_size = hh_size[np.cumsum(hh_size) < 1000]
    
    leftover = 1000 - hh_size.sum()
    if leftover < 3:
        # Randomly sample leftover amount indices from hh_size, and add one
        hh = np.arange(len(hh_size))
        idx = np.random.choice(hh[hh_size < 6], size = leftover, replace = False)
        hh_size[idx] += 1
    else:
        hh_size = np.append(hh_size, leftover)
    return hh_size

def SEIR(beta_H, beta_C, inc, inf, verbose = 0):
    hh_size = create_hh()
    
    # ID: ID of individual
    # SIZE: size of individual's household
    # HH: ID of individual's household
    # S: susceptibility status
    # E: exposed status
    # E_count: number of days since exposed
    # I: infectious status
    # I_count: number of days since infectious
    # R: recovered status
    # INC: incubation period
    # INF: infectious period
    # S_num: number of susceptible people in household when infectious period begins
    # I_num: number of people that this person infected over the infectious period
    data = pd.DataFrame({'ID': range(1000),
                         'SIZE': np.repeat(hh_size, repeats = hh_size),
                         'HH': np.repeat(range(len(hh_size)), repeats = hh_size),
                         'S': np.append(0, np.ones(999)),
                         'E': np.append(1, np.zeros(999)),
                         'E_count': np.append(1, np.zeros(999)),
                         'I': np.zeros(1000),
                         'I_count': np.zeros(1000),
                         'R': np.zeros(1000),
                         'INC': np.append(np.round(stats.norm.rvs(inc, 2)), np.zeros(999)),
                         'INF': np.zeros(1000)
                        })
    
    # Create frame for storing results
    results = data.loc[:, 'ID':'HH']
    results['TYPE'] = np.nan
    results.loc[0, 'TYPE'] = 'I' # index case is Type 0 
    results['TIME'] = np.nan
    #results['TIME_I'] = np.nan
    #results['TIME_E'] = np.nan
    results['S_num'] = np.nan
    results['I_num'] = 0
    
    for t in range(time):
        if verbose:
            if t % 10 == 0: print(t, end = ' ')
        
        # Anyone who has been infectious for as many days as their infectious
        # period is now recovered.
        recovered = (data['INF'] > 0) & (data['I_count'] == data['INF'])
        if sum(recovered) > 0:
            data.loc[recovered, 'R'] = 1
            data.loc[recovered, 'I'] = 0
            data.loc[recovered, 'I_count'] = 0

        # Anyone who has been incubating for as many days as their incubation
        # period is now infectious.
        new_inf = (data['INC'] > 0) & (data['E_count'] == data['INC'])
        num_new_inf = sum(new_inf)
        if num_new_inf > 0:
            random_inf = np.round(stats.norm.rvs(inf, 1, size = num_new_inf))
            data.loc[new_inf, 'I'] = 1
            data.loc[new_inf, 'INF'] = random_inf
            data.loc[new_inf, 'E'] = 0
            data.loc[new_inf, 'E_count'] = 0
            
            # If newly infected and there isn't already a time recorded, then update with the time t.
            results['TIME'].where(~(new_inf == 1) | ~pd.isna(results['TIME']), t, inplace = True)
            #results['TIME_I'].where(~(new_inf == 1) | ~pd.isna(results['TIME_I']), t, inplace = True)
            
            # Get the number of susceptible people in each household
            S_num = data.groupby('HH').sum()

            # Get the households of the newly infectious people
            hh_idx = data.loc[new_inf, :]['HH']
            
            # Save in results the number of susceptible people
            results.loc[new_inf, 'S_num'] = S_num.loc[hh_idx, 'S'].values
            
        # I_H is the number of infections in each household.
        # I_C is the number of infections outside a given household.
        I_H = data.groupby('HH').sum()['I']
        summary = pd.DataFrame({'I_H': I_H, 
                                'I_C': data.sum()['I'] - I_H
                               })
        # dd is a frame where each individual is assigned their household's
        # I_H and I_C numbers.
        dd = data[['HH', 'S']].copy()
        dd['I_H'] = dd.apply(lambda x: summary.loc[x['HH'], 'I_H'], axis = 1)
        dd['I_C'] = dd.apply(lambda x: summary.loc[x['HH'], 'I_C'], axis = 1)

        # Calculate household risk and community risk
        risk_H = dd['S'] * beta_H * dd['I_H'] / 1000
        risk_C = dd['S'] * beta_C * dd['I_C'] / 1000

        # Calculate new household and community infections
        new_inf_H = stats.binom.rvs(1, risk_H, size = data.shape[0])
        new_inf_C = stats.binom.rvs(1, risk_C, size = data.shape[0])
        new_exposed = (new_inf_H == 1) | (new_inf_C == 1)

        num_new_exposed = sum(new_exposed)
        if num_new_exposed > 0:
            data.loc[new_exposed, 'E'] = 1
            random_inc = np.round(stats.norm.rvs(inc, 2, size = num_new_exposed))
            data.loc[new_exposed, 'E'] = 1
            data.loc[new_exposed, 'INC'] = random_inc
            
            #print(np.sum( (new_inf_H == 1) & (~pd.isna(results['TYPE'])) ))
            # The NA checks may not be necessary, since S = 0 for anyone who already has been assigned a TYPE
            results['TYPE'].where(~(new_inf_H == 1) | ~pd.isna(results['TYPE']), 'H', inplace = True)
            results['TYPE'].where(~(new_inf_C == 1) | ~pd.isna(results['TYPE']), 'C', inplace = True)
            #results['TIME_E'].where(~(new_exposed == 1) | ~pd.isna(results['TIME_E']), t, inplace = True)
            
            # Number of new infections in each household
            rr = results.loc[new_inf_H == 1, :].groupby('HH')['TYPE'].count()
            
            # Get individuals with the smallest infectious counts
            ids = data[data['I'] == 1].groupby('HH').min()['ID']
            
            # Increase I_num by the amount of people in household that got a H-type infection
            res_idx = results['ID'].isin(ids) & (results['HH'].isin(rr.index))
            results.loc[res_idx, 'I_num'] += rr.values
            
            # After the above analysis, assign 'B' to people who were infected by both.
            results['TYPE'].where(~((new_inf_H == 1) & (new_inf_C == 1)) | ~pd.isna(results['TYPE']), 'B', inplace = True)
            
        data.loc[data['E'] == 1, 'E_count'] += 1
        data.loc[data['I'] == 1, 'I_count'] += 1
        data.loc[data['E'] == 1, 'S'] = 0

    return results

def metrics(results):
    state = results[~pd.isna(results['TIME'])]
    idc = state.shape[0]/1000
    
    sar = np.nan
    if idc != 0:
        sar = np.mean(results['I_num'] / results['S_num'])
    
    return idc, sar

def score(results, target):
    return np.sum((np.array(metrics(results)) - target)**2)

In [ ]:
beta_H = hh[20][20]
beta_C = cc[10][10]

In [ ]:
beta_H

In [ ]:
beta_C

In [ ]:
results = SEIR(beta_H, beta_C, inc, inf)

In [ ]:
metrics(results)

## Variance Check

### Case 1

For Case 1, 
- $\beta_H = 30$;
- $\beta_C = 0.12$.

In [ ]:
idcs_10 = np.load('results/8/idcs_1_10.npy')
idcs_50 = np.load('results/8/idcs_1_50.npy')
idcs_100 = np.load('results/8/idcs_1_100.npy')
idcs_250 = np.load('results/8/idcs_1_250.npy')
idcs_500 = np.load('results/8/idcs_1_500.npy')
idcs_750 = np.load('results/8/idcs_1_750.npy')
idcs_1000 = np.load('results/8/idcs_1_1000.npy')

sars_10 = np.load('results/8/sars_1_10.npy')
sars_50 = np.load('results/8/sars_1_50.npy')
sars_100 = np.load('results/8/sars_1_100.npy')
sars_250 = np.load('results/8/sars_1_250.npy')
sars_500 = np.load('results/8/sars_1_500.npy')
sars_750 = np.load('results/8/sars_1_750.npy')
sars_1000 = np.load('results/8/sars_1_1000.npy')

n_10 = np.arange(1, 11)
n_50 = np.arange(1, 51)
n_100 = np.arange(1, 101)
n_250 = np.arange(1, 251)
n_500 = np.arange(1, 501)
n_750 = np.arange(1, 751)
n_1000 = np.arange(1, 1001)

In [ ]:
idc_avg_10 = np.cumsum(idcs_10) / n_10
idc_avg_50 = np.cumsum(idcs_50) / n_50
idc_avg_100 = np.cumsum(idcs_100) / n_100
idc_avg_250 = np.cumsum(idcs_250) / n_250
idc_avg_500 = np.cumsum(idcs_500) / n_500
idc_avg_750 = np.cumsum(idcs_750) / n_750
idc_avg_1000 = np.cumsum(idcs_1000) / n_1000

sar_avg_10 = np.cumsum(sars_10) / n_10
sar_avg_50 = np.cumsum(sars_50) / n_50
sar_avg_100 = np.cumsum(sars_100) / n_100
sar_avg_250 = np.cumsum(sars_250) / n_250
sar_avg_500 = np.cumsum(sars_500) / n_500
sar_avg_750 = np.cumsum(sars_750) / n_750
sar_avg_1000 = np.cumsum(sars_1000) / n_1000

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (11, 3.5))

axs[0].plot(n_10, idc_avg_10, alpha = 0.75)
axs[0].plot(n_50, idc_avg_50, alpha = 0.75)
axs[0].plot(n_100, idc_avg_100, alpha = 0.75)
axs[0].plot(n_250, idc_avg_250, alpha = 0.75)
axs[0].plot(n_500, idc_avg_500, alpha = 0.75)
axs[0].plot(n_750, idc_avg_750, alpha = 0.75)
axs[0].plot(n_1000, idc_avg_1000, alpha = 0.75)
axs[0].set_xlabel('Number of Simulations')
axs[0].set_ylabel('Average Incidence')
axs[0].set_title(r'$\beta_H = 30$, $\beta_C = 0.12$', loc = 'left')
axs[0].set_title(r'Convergence of Incidence', loc = 'left')

axs[1].plot(n_10, sar_avg_10, alpha = 0.75)
axs[1].plot(n_50, sar_avg_50, alpha = 0.75)
axs[1].plot(n_100, sar_avg_100, alpha = 0.75)
axs[1].plot(n_250, sar_avg_250, alpha = 0.75)
axs[1].plot(n_500, sar_avg_500, alpha = 0.75)
axs[1].plot(n_750, sar_avg_750, alpha = 0.75)
axs[1].plot(n_1000, sar_avg_1000, alpha = 0.75)
axs[1].set_xlabel('Number of Simulations')
axs[1].set_ylabel('Average SAR')
axs[1].set_title(r'$\beta_H = 30$, $\beta_C = 0.12$', loc = 'left')
axs[1].set_title(r'Convergence of SAR', loc = 'left')

plt.savefig('plots/case_1.png', bbox_inches = 'tight', dpi = 300)

### Case 2

For Case 2, 
- $\beta_H = 23$;
- $\beta_C = 0.1$.

In [ ]:
idcs_10 = np.load('results/8/idcs_2_10.npy')
idcs_50 = np.load('results/8/idcs_2_50.npy')
idcs_100 = np.load('results/8/idcs_2_100.npy')
idcs_250 = np.load('results/8/idcs_2_250.npy')
idcs_500 = np.load('results/8/idcs_2_500.npy')
idcs_750 = np.load('results/8/idcs_2_750.npy')
idcs_1000 = np.load('results/8/idcs_2_1000.npy')

sars_10 = np.load('results/8/sars_2_10.npy')
sars_50 = np.load('results/8/sars_2_50.npy')
sars_100 = np.load('results/8/sars_2_100.npy')
sars_250 = np.load('results/8/sars_2_250.npy')
sars_500 = np.load('results/8/sars_2_500.npy')
sars_750 = np.load('results/8/sars_2_750.npy')
sars_1000 = np.load('results/8/sars_2_1000.npy')

n_10 = np.arange(1, 11)
n_50 = np.arange(1, 51)
n_100 = np.arange(1, 101)
n_250 = np.arange(1, 251)
n_500 = np.arange(1, 501)
n_750 = np.arange(1, 751)
n_1000 = np.arange(1, 1001)

In [ ]:
idc_avg_10 = np.cumsum(idcs_10) / n_10
idc_avg_50 = np.cumsum(idcs_50) / n_50
idc_avg_100 = np.cumsum(idcs_100) / n_100
idc_avg_250 = np.cumsum(idcs_250) / n_250
idc_avg_500 = np.cumsum(idcs_500) / n_500
idc_avg_750 = np.cumsum(idcs_750) / n_750
idc_avg_1000 = np.cumsum(idcs_1000) / n_1000

sar_avg_10 = np.cumsum(sars_10) / n_10
sar_avg_50 = np.cumsum(sars_50) / n_50
sar_avg_100 = np.cumsum(sars_100) / n_100
sar_avg_250 = np.cumsum(sars_250) / n_250
sar_avg_500 = np.cumsum(sars_500) / n_500
sar_avg_750 = np.cumsum(sars_750) / n_750
sar_avg_1000 = np.cumsum(sars_1000) / n_1000

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (11, 3.5))

axs[0].plot(n_10, idc_avg_10, alpha = 0.75)
axs[0].plot(n_50, idc_avg_50, alpha = 0.75)
axs[0].plot(n_100, idc_avg_100, alpha = 0.75)
axs[0].plot(n_250, idc_avg_250, alpha = 0.75)
axs[0].plot(n_500, idc_avg_500, alpha = 0.75)
axs[0].plot(n_750, idc_avg_750, alpha = 0.75)
axs[0].plot(n_1000, idc_avg_1000, alpha = 0.75)
axs[0].set_xlabel('Number of Simulations')
axs[0].set_ylabel('Average Incidence')
axs[0].set_title(r'$\beta_H = 23$, $\beta_C = 0.1$', loc = 'left')
axs[0].set_title(r'Convergence of Incidence', loc = 'left')

axs[1].plot(n_10, sar_avg_10, alpha = 0.75)
axs[1].plot(n_50, sar_avg_50, alpha = 0.75)
axs[1].plot(n_100, sar_avg_100, alpha = 0.75)
axs[1].plot(n_250, sar_avg_250, alpha = 0.75)
axs[1].plot(n_500, sar_avg_500, alpha = 0.75)
axs[1].plot(n_750, sar_avg_750, alpha = 0.75)
axs[1].plot(n_1000, sar_avg_1000, alpha = 0.75)
axs[1].set_xlabel('Number of Simulations')
axs[1].set_ylabel('Average SAR')
axs[1].set_title(r'$\beta_H = 23$, $\beta_C = 0.1$', loc = 'left')
axs[1].set_title(r'Convergence of SAR', loc = 'left')

plt.savefig('plots/case_2.png', bbox_inches = 'tight', dpi = 300)

### Case 3

For Case 3, 
- $\beta_H = 11$;
- $\beta_C = 0.05$.

In [ ]:
idcs_10 = np.load('results/8/idcs_3_10.npy')
idcs_50 = np.load('results/8/idcs_3_50.npy')
idcs_100 = np.load('results/8/idcs_3_100.npy')
idcs_250 = np.load('results/8/idcs_3_250.npy')
idcs_500 = np.load('results/8/idcs_3_500.npy')
idcs_750 = np.load('results/8/idcs_3_750.npy')
idcs_1000 = np.load('results/8/idcs_3_1000.npy')

sars_10 = np.load('results/8/sars_3_10.npy')
sars_50 = np.load('results/8/sars_3_50.npy')
sars_100 = np.load('results/8/sars_3_100.npy')
sars_250 = np.load('results/8/sars_3_250.npy')
sars_500 = np.load('results/8/sars_3_500.npy')
sars_750 = np.load('results/8/sars_3_750.npy')
sars_1000 = np.load('results/8/sars_3_1000.npy')

n_10 = np.arange(1, 11)
n_50 = np.arange(1, 51)
n_100 = np.arange(1, 101)
n_250 = np.arange(1, 251)
n_500 = np.arange(1, 501)
n_750 = np.arange(1, 751)
n_1000 = np.arange(1, 1001)

In [ ]:
idc_avg_10 = np.cumsum(idcs_10) / n_10
idc_avg_50 = np.cumsum(idcs_50) / n_50
idc_avg_100 = np.cumsum(idcs_100) / n_100
idc_avg_250 = np.cumsum(idcs_250) / n_250
idc_avg_500 = np.cumsum(idcs_500) / n_500
idc_avg_750 = np.cumsum(idcs_750) / n_750
idc_avg_1000 = np.cumsum(idcs_1000) / n_1000

sar_avg_10 = np.cumsum(sars_10) / n_10
sar_avg_50 = np.cumsum(sars_50) / n_50
sar_avg_100 = np.cumsum(sars_100) / n_100
sar_avg_250 = np.cumsum(sars_250) / n_250
sar_avg_500 = np.cumsum(sars_500) / n_500
sar_avg_750 = np.cumsum(sars_750) / n_750
sar_avg_1000 = np.cumsum(sars_1000) / n_1000

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (11, 3.5))

axs[0].plot(n_10, idc_avg_10, alpha = 0.75)
axs[0].plot(n_50, idc_avg_50, alpha = 0.75)
axs[0].plot(n_100, idc_avg_100, alpha = 0.75)
axs[0].plot(n_250, idc_avg_250, alpha = 0.75)
axs[0].plot(n_500, idc_avg_500, alpha = 0.75)
axs[0].plot(n_750, idc_avg_750, alpha = 0.75)
axs[0].plot(n_1000, idc_avg_1000, alpha = 0.75)
axs[0].set_xlabel('Number of Simulations')
axs[0].set_ylabel('Average Incidence')
axs[0].set_title(r'$\beta_H = 11$, $\beta_C = 0.05$', loc = 'left')
axs[0].set_title(r'Convergence of Incidence', loc = 'left')

axs[1].plot(n_10, sar_avg_10, alpha = 0.75)
axs[1].plot(n_50, sar_avg_50, alpha = 0.75)
axs[1].plot(n_100, sar_avg_100, alpha = 0.75)
axs[1].plot(n_250, sar_avg_250, alpha = 0.75)
axs[1].plot(n_500, sar_avg_500, alpha = 0.75)
axs[1].plot(n_750, sar_avg_750, alpha = 0.75)
axs[1].plot(n_1000, sar_avg_1000, alpha = 0.75)
axs[1].set_xlabel('Number of Simulations')
axs[1].set_ylabel('Average SAR')
axs[1].set_title(r'$\beta_H = 11$, $\beta_C = 0.05$', loc = 'right');
axs[1].set_title(r'Convergence of SAR', loc = 'left')

plt.savefig('plots/case_3.png', bbox_inches = 'tight', dpi = 300)

### Scores

In [ ]:
idcs_1_1000 = np.load('results/8/idcs_1_1000.npy')
sars_1_1000 = np.load('results/8/sars_1_1000.npy')
vals_1 = np.vstack([idcs_1_1000, sars_1_1000]).T

idcs_2_1000 = np.load('results/8/idcs_2_1000.npy')
sars_2_1000 = np.load('results/8/sars_2_1000.npy')
vals_2 = np.vstack([idcs_2_1000, sars_2_1000]).T

idcs_3_1000 = np.load('results/8/idcs_3_1000.npy')
sars_3_1000 = np.load('results/8/sars_3_1000.npy')
vals_3 = np.vstack([idcs_3_1000, sars_3_1000]).T

In [ ]:
target = np.repeat(np.array([[0.3, 0.25]]), 1000, axis = 0)

In [ ]:
idc_1_avgs = np.cumsum(vals_1[:, 0]) / n_1000
sar_1_avgs = np.cumsum(vals_1[:, 1]) / n_1000

idc_2_avgs = np.cumsum(vals_2[:, 0]) / n_1000
sar_2_avgs = np.cumsum(vals_2[:, 1]) / n_1000

idc_3_avgs = np.cumsum(vals_3[:, 0]) / n_1000
sar_3_avgs = np.cumsum(vals_3[:, 1]) / n_1000

scores_1 = (idc_1_avgs - 0.3)**2 + (sar_1_avgs - 0.25)**2
scores_2 = (idc_2_avgs - 0.3)**2 + (sar_2_avgs - 0.25)**2
scores_3 = (idc_3_avgs - 0.3)**2 + (sar_3_avgs - 0.25)**2

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (11, 3.5))

axs[0].plot(n_1000, scores_1, label = r'$\beta_H = 30$, $\beta_C = 0.12$')
axs[0].plot(n_1000, scores_2, label = r'$\beta_H = 23$, $\beta_C = 0.1$')
axs[0].plot(n_1000, scores_3, label = r'$\beta_H = 11$, $\beta_C = 0.05$')
axs[0].set_xlabel('Number of Simulations')
axs[0].set_ylabel(r'$\ell^2$ Score')
axs[0].set_title('Convergence of Score')
axs[0].legend()

axs[1].plot(n_1000, -np.log(scores_1), label = r'$\beta_H = 30$, $\beta_C = 0.12$')
axs[1].plot(n_1000, -np.log(scores_2), label = r'$\beta_H = 23$, $\beta_C = 0.1$')
axs[1].plot(n_1000, -np.log(scores_3), label = r'$\beta_H = 11$, $\beta_C = 0.05$')
axs[1].set_xlabel('Number of Simulations')
axs[1].set_ylabel(r'Likelihood')
axs[1].set_title('Convergence of Likelihood')
axs[1].legend()

plt.savefig('plots/score.png',  bbox_inches = 'tight', dpi = 300)

# 17 March

In [ ]:
vals_r = pd.read_csv('validation/vals.csv').drop(['Unnamed: 0'], axis = 1)

In [ ]:
vals_r

In [ ]:
vals_r.

In [ ]:
plt.hist(vals_r['V1'], alpha = 0.5)

In [ ]:
plt.hist(vals_r['V2'], alpha = 0.5)

In [ ]:
np.mean(vals_r['V2'])

In [ ]:
np.mean(vals_r['V1'])

# 20 March

In [ ]:
idcs = np.load('results/10/idcs.npy')
sars = np.load('results/10/sars.npy')

cc = np.load('results/10/cc.npy')
hh = np.load('results/10/hh.npy')

In [ ]:
mean_idcs = idcs.mean(axis = 2)
mean_sars = sars.mean(axis = 2)

In [ ]:
cc[mean_sars > 0.2]

In [ ]:
hh[mean_sars > 0.2]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize = (11, 3.5))

plot_hh = axs[0].scatter(mean_idcs.flatten(), mean_sars.flatten(), 
                         c = hh.flatten(), alpha = 0.5)

axs[0].set_xlabel(r'$I$')
axs[0].set_ylabel(r'${\rm SAR}$')
#axs[0].set_aspect('equal')
fig.colorbar(plot_hh, ax = axs[0])
axs[0].set_title(r'$\beta_H$', loc = 'left')
axs[0].scatter(0.05, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.1, 0.25, marker = 'x', c = 'red', s = 100)
axs[0].scatter(0.3, 0.25, marker = 'x', c = 'red', s = 100)

plot_cc = axs[1].scatter(mean_idcs.flatten(), mean_sars.flatten(), 
                         c = cc.flatten(), alpha = 0.5)
axs[1].set_xlabel(r'$I$')
axs[1].set_ylabel(r'${\rm SAR}$')
#axs[1].set_aspect('equal')
fig.colorbar(plot_hh, ax = axs[1])
axs[1].set_title(r'$\beta_C$', loc = 'left')
axs[1].scatter(0.05, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.1, 0.25, marker = 'x', c = 'red', s = 100)
axs[1].scatter(0.3, 0.25, marker = 'x', c = 'red', s = 100)

plt.savefig('plots/10.png', transparent = True, dpi = 300)

In [ ]:
cc[(mean_idcs >= .09) & (mean_idcs <= .11)]

In [ ]:
hh[(mean_idcs >= .09) & (mean_idcs <= .11)]

In [9]:
arr = np.loadtxt('idcs.txt')
arr

array([[0.005, 0.002, 0.005, 0.002, 0.006, 0.001, 0.002, 0.002, 0.001],
       [0.994, 0.996, 0.998, 0.985, 0.994, 0.997, 0.982, 0.993, 0.992],
       [1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   , 1.   ]])

In [14]:
arr = arr.reshape(3, 3, 3)

In [17]:
arr

array([[[0.005, 0.002, 0.005],
        [0.002, 0.006, 0.001],
        [0.002, 0.002, 0.001]],

       [[0.994, 0.996, 0.998],
        [0.985, 0.994, 0.997],
        [0.982, 0.993, 0.992]],

       [[1.   , 1.   , 1.   ],
        [1.   , 1.   , 1.   ],
        [1.   , 1.   , 1.   ]]])

In [21]:
arr.mean(axis = 2)

array([[0.004     , 0.003     , 0.00166667],
       [0.996     , 0.992     , 0.989     ],
       [1.        , 1.        , 1.        ]])

In [12]:
cc, hh = np.meshgrid([35, 50, 65], [0, 0.5, 1])

In [13]:
cc

array([[35, 50, 65],
       [35, 50, 65],
       [35, 50, 65]])